In [1]:
from flask import Flask, request
import pickle
import numpy as np
import torch
import cv2
import io
from matplotlib import pyplot as plt
from position_calculator import calculate_position 
#The Python Imaging Library adds image processing capabilities to your Python interpreter. 
import PIL
#Pickle is used for serializing and de-serializing Python object structures, 
#also called marshalling or flattening. Serialization refers to the process 
#of converting an object in memory to a byte stream that can be stored on 
#disk or sent over a network. Later on, this character stream can then be 
#retrieved and de-serialized back to a Python object. 
#Unpickling a file that was pickled in a different version of Python may 
#not always work properly
#https://www.datacamp.com/tutorial/pickle-python-tutorial

In [2]:
# This dictionary will be utilized while finding the distance of objects from the user. 
# We are using the width of an object instead of its height because while capturing the 
# frame, the complete height of the object might not be captured. The width is more likely
# to be appearing in full length.
#'motorcycle': ,'traffic light': , 'bench': ,'dog': , 'cat': ,  'bed': , 'dining_table': , 
objects_actual_width = {'person': 38.5, 'tv':5, 'bicycle': 175, 'couch': 152, 'bus': 1200, 'car': 448, 'chair': 46}

In [3]:
navi_app = Flask(__name__)
# after the application starts, we can load the model for once 
# and make several requests to it
f = open("object_detection-v1.pk", "rb") # reading a binary file
model_pk = pickle.load(f) # loading it as a model object

In [4]:
@navi_app.route('/api_predict', methods=["GET", "POST"])
def api_predict():
    if request.method == "GET":
        return "Please send post request"

    elif request.method == "POST":
        frame = request.files.get('frame') # get the frame sent by the API request 
        im_bytes = frame.read() # convert the file into byte stream
        image = PIL.Image.open(io.BytesIO(im_bytes)) # convert the byte stream into 
        image_width, image_height = image.size
        
        prediction = model_pk(image)
        objects_with_positions = calculate_position(prediction.pandas().xyxy, objects_actual_width, image_width, image_height)
        
        return objects_with_positions

In [ ]:
navi_app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [17/Dec/2022 13:08:02] "POST /api_predict HTTP/1.1" 200 -
127.0.0.1 - - [17/Dec/2022 13:08:33] "POST /api_predict HTTP/1.1" 200 -
